In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import random
import time


for ID in range(200):

    info = {
        'White Percentage': [],
        'Stretch Coords': [],
        'Squeeze Coords': [],
        'Vertical Coords Start': [],
        'Vertical Coords End': [],
        'Horizontal Coords Start': [],
        'Horizontal Coords End': [],
        'Diagonal Coords Start': [],
        'Diagonal Coords End': [],
        'Combination Coords Start': [],
        'Combination Coords End': []
    }

    #working generate pixels and coordinates start from bottom left
    d = {'IMG_ID': [], 'PIXEL_X': [], 'PIXEL_Y': [], 'PIXEL_COLOR': []}
    df = pd.DataFrame(data=d)

    df['PIXEL_X'] = np.tile(range(112), 69)
    df['PIXEL_Y'] = np.repeat(range(69), 112)

    num_pixels = 112 * 69
    white_percentage = random.randint(1, 2) /100
    info['White Percentage'].append(white_percentage)
    num_white_pixels = int(white_percentage * num_pixels)
    df['PIXEL_COLOR'] = np.concatenate([np.zeros(num_pixels - num_white_pixels), 
                                        np.ones(num_white_pixels)])

    def check_3pixel():
        while True:
            df['PIXEL_COLOR'] = np.random.permutation(df['PIXEL_COLOR'])
            found = 0
            for y in range(69):
                for x in range(110):
                    selected_values = df.loc[(df['PIXEL_X'].between(x, x+2)) & 
                                    (df['PIXEL_Y'] == y), 'PIXEL_COLOR']
                    if selected_values.nunique() == 1.0:
                        if selected_values.unique() == 1.0:
                            found += 1

            for y in range(67):
                for x in range(112):
                    selected_values = df.loc[(df['PIXEL_Y'].between(y, y+2)) & 
                                    (df['PIXEL_X'] == x), 'PIXEL_COLOR']
                    if selected_values.nunique() == 1.0:
                        if selected_values.unique() == 1.0:
                            found += 1

            for y in range(67):
                for x in range(110):
                    selected_values = df.loc[(df['PIXEL_X'].between(x, x+2)) & 
                                    (df['PIXEL_Y'].between(y, y+2)) &
                                    (df['PIXEL_X'] - df['PIXEL_Y'] == x - y), 'PIXEL_COLOR']
                    if selected_values.nunique() == 1.0:
                        if selected_values.unique() == 1.0:
                            found += 1

            for y in range(67):
                for x in range(110):
                    selected_values = df.loc[(df['PIXEL_X'].between(x, x+2)) & 
                                    (df['PIXEL_Y'].between(y, y+2)) & 
                                    (df['PIXEL_X'] + df['PIXEL_Y'] == x + y + 2), 'PIXEL_COLOR']

                    if selected_values.nunique() == 1.0:
                        if selected_values.unique() == 1.0:
                            found += 1
            print(found)
            if found == 0:
                break

    check_3pixel()



    def generate_semiconductor_image(size_x, size_y, center, dispersion_factor, starting_intensity_factor):
        # Create a grid of coordinates
        x, y = np.meshgrid(np.arange(size_x), np.arange(size_y))
        
        # Calculate distances from the center
        distances = np.sqrt((x - center[0])**2 + (y - center[1])**2)
        
        # Generate pixel values based on a Gaussian distribution
        intensity = np.exp(-(distances / dispersion_factor)**2)
        
        # Normalize pixel values to be between 0 and 1
        intensity = intensity / np.max(intensity)
        
        # Adjust starting intensity
        intensity *= starting_intensity_factor
        
        # Generate the synthetic image
        image = np.random.rand(size_y, size_x) < intensity
        
        # Reshape the 2D array into a 1D array
        flat_image = image.flatten()
        
        # Create a DataFrame with a single column named 'Pixel'
        binary_array = flat_image.astype(int)
        df['PIXEL_COLOR'] = np.logical_or(df['PIXEL_COLOR'].values, binary_array).astype(int)
        
        return df

    image_size_x = 112
    image_size_y = 69

    def generate_stm_nolasso_amt():
        stm_nolasso_count = []
        while True:
            # Set probabilities for stretch and squeeze values
            stretch_probabilities = [0.1, 0.45, 0.45]  # Lower probability for the first value
            squeeze_probabilities = [0.2, 0.3, 0.3, 0.2]  # Lower probability for the first value

            # Generate random values for stretch and squeeze using numpy.random.choice
            stretch = np.random.choice([0, 1, 2], p=stretch_probabilities)
            squeeze = np.random.choice([0, 1, 2, 3], p=squeeze_probabilities)

            if not ((stretch + squeeze) > 3):
                stm_nolasso_count.append(stretch)
                stm_nolasso_count.append(squeeze)
                break
        return stm_nolasso_count
        

    stm_nolasso_count = generate_stm_nolasso_amt()


    if stm_nolasso_count[0] > 0:
        for i in range(stm_nolasso_count[0]):
        #d20,i0.3
            dispersion_factor = 20
            starting_intensity_factor = 0.3
            while True:
                center_x = random.randint(-20,131)
                center_y = random.randint(-20,88)
                if not ((1 < center_x < 110) & (1 < center_y < 67)):
                    center = (center_x, center_y)
                    print(center)
                    info['Stretch Coords'].append(center)
                    df = generate_semiconductor_image(image_size_x, image_size_y, center, dispersion_factor, starting_intensity_factor)
                    break

    if stm_nolasso_count[1] > 0:
        for i in range(stm_nolasso_count[1]):
        #d10,i0.5
            dispersion_factor = 10
            starting_intensity_factor = 0.5
            while True:
                center_x = random.randint(-5,116)
                center_y = random.randint(-5,73)
                if not (((3 < center_x < 108) & (3 < center_y < 65))) | (((-5 <= center_x < 0) & (-5 <= center_y < 0))) | (((-5 <= center_x < 0) & (68 < center_y <= 73))) | (((111 < center_x <= 116) & (-5 <= center_y < 0))) | (((111 < center_x <= 116) & (68 < center_y <= 73))):
                    center = (center_x, center_y)
                    print(center)
                    info['Squeeze Coords'].append(center)
                    df = generate_semiconductor_image(image_size_x, image_size_y, center, dispersion_factor, starting_intensity_factor)
                    break



    # Display the DataFrame
    df = pd.DataFrame(np.nan_to_num(df, nan=(ID+1)), columns=df.columns)
    df = df.astype(int)
    df.to_csv(f'no_lasso_200/CSV_data/IMG_ID{ID+1}.csv', index=False)

    image = df['PIXEL_COLOR'].values.reshape(69, 112)
    plt.rc('axes',edgecolor='w')

    # Create a square figure with subplots
    fig, ax = plt.subplots(figsize=(11.5, 10.5))

    # Plot the image on the single subplot
    ax.imshow(image, cmap='gray', interpolation='nearest', origin='lower')

    # Set the aspect ratio to 'auto'
    ax.set_aspect('auto', adjustable='box')
    ax.tick_params('both', length=7, width=2, which='major')

    plt.xticks(np.arange(0, 112, 4), rotation=90)
    plt.yticks(np.arange(0, 68, 4))
    plt.xlabel('PIXEL_X', fontsize=20, labelpad=10)
    plt.ylabel('PIXEL_Y', fontsize=20, labelpad=10)
    plt.title(f"IMG_ID: {ID+1}", fontsize=20, pad=15)

    plt.savefig(f'no_lasso_200/IMG/IMG_ID{ID+1}.png')

    for key in info:
        if len(info[key]) < 3:
            info[key].extend([' '] * (3 - len(info[key])))
    df_info = pd.DataFrame(info)

    df_info.to_csv(f'no_lasso_200/CSV_info/IMG_ID{ID+1}_info.csv', index=False)

    plt.show()

    time.sleep(1)   

